Installing required Libraries

In [ ]:
!pip install git+https://${GITHUB_TOKEN}@github.com/planetlabs/maxrect.git
!pip install shapely
!pip install CVXPY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The Following Cells extract the path of csv files and tmc files from the drive

In [ ]:
import os

In [ ]:
paths = os.path.join('/content/drive/MyDrive/OHRC Files/ohrc files',i)

In [ ]:
img_paths = []
xml_paths = []
number_o = []
csv_paths = []
new_path2 = []
base = '/content/drive/MyDrive/OHRC_Files/ohrc_files'
files = os.listdir('/content/drive/MyDrive/OHRC_Files/ohrc_files')
for i in files:
    path = os.path.join(base,i,'data','calibrated')
    new_path2.append(i)
    pth = os.listdir(path)
    data_path = os.path.join(path,pth[0])
    data_content = os.listdir(data_path)
    img_paths.append(data_content[0])
    xml_paths.append(data_content[1])
    paths1 = os.path.join(base,i,'geometry','calibrated')
    pth1 = os.listdir(paths1)
    number_o.append(pth1)
    geo_path1 = os.path.join(paths1,pth1[0])
    geo_content1 = os.listdir(geo_path1)
    csv_paths.append(geo_content1[0])

In [ ]:
import pandas as pd
df_ohrc = pd.DataFrame({
        'img_path':img_paths,
        'xml_path':xml_paths
})

In [ ]:
df_ohrc['csv_path'] = csv_paths
df_ohrc['Number'] = number_o
df_ohrc['New_Path2'] = new_path2

In [ ]:
img_paths = []
xml_paths = []
csv_paths = []
number = []
new_path = []
base_1 = '/content/drive/MyDrive/TMC_Files/Tmc_data'
files_1 = os.listdir('/content/drive/MyDrive/TMC_Files/Tmc_data')
for i in files_1:
  try:
      path = os.path.join(base_1,i,'data','calibrated')
      pth = os.listdir(path)
      data_path = os.path.join(path,pth[0])
      data_content = os.listdir(data_path)
      paths1 = os.path.join(base_1,i,'geometry','calibrated')
      pth1 = os.listdir(paths1)
      geo_path1 = os.path.join(paths1,pth1[0])
      geo_content1 = os.listdir(geo_path1)
      if len(data_content)>=2:
        new_path.append(i)
        number.append(pth1)
        csv_paths.append(geo_content1[0])
        img_paths.append(data_content[0])
        # print(data_content)
        xml_paths.append(data_content[1])
  except:
      pass

In [ ]:
import pandas as pd
df_tmc = pd.DataFrame({
        'img_path':img_paths,
        'xml_path':xml_paths
})
df_tmc['csv_path'] = csv_paths
df_tmc['Number'] = number
df_tmc['new_path'] = new_path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import numpy as np
import cvxpy
from shapely.geometry import Polygon

Gives coordinats of rectange given upper right and lower left corner coordinates

In [ ]:
def rect2poly(ll, ur):
    x0, y0 = ll
    x1, y1 = ur

    return [
        [x0, y0],
        [x0, y1],
        [x1, y1],
        [x1, y0],
        [x0, y0]
    ]

Gives the coordinates of largest rectangle inside a polygon given coordinates of its boundary points

In [ ]:
def get_maximal_rectangle(coordinates):
    coordinates = np.array(coordinates)
    x_range = np.max(coordinates, axis=0)[0]-np.min(coordinates, axis=0)[0]
    y_range = np.max(coordinates, axis=0)[1]-np.min(coordinates, axis=0)[1]

    scale = np.array([x_range, y_range])
    sc_coordinates = coordinates/scale

    poly = Polygon(sc_coordinates)
    inside_pt = (poly.representative_point().x,
                 poly.representative_point().y)

    A1, A2, B = pts_to_leq(sc_coordinates)

    bl = cvxpy.Variable(2)
    tr = cvxpy.Variable(2)
    br = cvxpy.Variable(2)
    tl = cvxpy.Variable(2)
    obj = cvxpy.Maximize(cvxpy.log(tr[0] - bl[0]) + cvxpy.log(tr[1] - bl[1]))
    constraints = [bl[0] == tl[0],
                   br[0] == tr[0],
                   tl[1] == tr[1],
                   bl[1] == br[1],
                   ]

    for i in range(len(B)):
        if inside_pt[0] * A1[i] + inside_pt[1] * A2[i] <= B[i]:
            constraints.append(bl[0] * A1[i] + bl[1] * A2[i] <= B[i])
            constraints.append(tr[0] * A1[i] + tr[1] * A2[i] <= B[i])
            constraints.append(br[0] * A1[i] + br[1] * A2[i] <= B[i])
            constraints.append(tl[0] * A1[i] + tl[1] * A2[i] <= B[i])

        else:
            constraints.append(bl[0] * A1[i] + bl[1] * A2[i] >= B[i])
            constraints.append(tr[0] * A1[i] + tr[1] * A2[i] >= B[i])
            constraints.append(br[0] * A1[i] + br[1] * A2[i] >= B[i])
            constraints.append(tl[0] * A1[i] + tl[1] * A2[i] >= B[i])

    prob = cvxpy.Problem(obj, constraints)
    prob.solve()

    bottom_left = np.array(bl.value).T * scale
    top_right = np.array(tr.value).T * scale

    return list(bottom_left), list(top_right)

returns diatance between 2 points

In [ ]:
def distance(a,b):
    dist = (a[0]-b[0])**2 + (a[1]-b[1])**2

Returns Max values of pixel and scan from csv files of tmc or 
ohrc data

In [ ]:
def get_max(csv):
    pixel = csv.Pixel.max()
    scan = csv.Scan.max()

    return pixel,scan

Returns latitude and longitude given pixel and scan value

In [ ]:
def get_lat_long(pixel,scan,csv):
    x1 = csv[(csv.Pixel==pixel) & (csv.Scan==scan)]['Longitude'].item()
    y1 = csv[(csv.Pixel==pixel) & (csv.Scan==scan)]['Latitude'].item()

    return x1,y1

Returns Boundary points of a imgaes in terms of latitude and longitude

In [ ]:
def bound(csv):
    pixel,scan = get_max(csv)
    bound = [get_lat_long(0,0,csv),get_lat_long(pixel,0,csv),get_lat_long(pixel,scan,csv),get_lat_long(0,scan,csv)]

    # bound.append(bound[0])
    # lr,ur = get_maximal_rectangle(bound)
    # bound = rect2poly(lr,ur)
    # points = bound[:len(bound)-1]

    return bound

Returns line equation parameters a,b,c
"aX + bY = c"

In [ ]:
def line(point1,point2):
    a = point2[1]-point1[1]
    b = point2[0] - point1[0]
    c  = point1[1]*b-point1[0]*a

    return a,-b,-c

The below two functions check whether two quadilaterls overlap or not given there boundary points

In [ ]:
def custom(line_points,points_tmc,other_ohrc_points):
    a,b,c = line(line_points[0],line_points[1])
    t = []
    o = []

    for i in points_tmc:
        temp = a*i[0] + b*i[1] - c
        t.append(temp)

    for i in other_ohrc_points:
        temp = a*i[0] + b*i[1] - c
        o.append(temp)

    t1,t2 = min(t),max(t)
    for t in t:
        if t==0:
            break
    else:
        if t1*t2 >= 0 >= t1*o[1]:
            return False
        else:
            return True
    return True

In [ ]:
def overlap_check(ohrc_boundary, tmc_boundary):
    ohrc_grp1 = (ohrc_boundary[0],ohrc_boundary[1])
    ohrc_grp2 = (ohrc_boundary[2],ohrc_boundary[3])
    ohrc_grp3 = (ohrc_boundary[0], ohrc_boundary[3])
    ohrc_grp4 = (ohrc_boundary[1], ohrc_boundary[2])

    if custom(ohrc_grp1, tmc_boundary, ohrc_grp2) and custom(ohrc_grp2, tmc_boundary, ohrc_grp1) and custom(ohrc_grp3, tmc_boundary, ohrc_grp4) and custom(ohrc_grp4, tmc_boundary, ohrc_grp3):
      print("Overlap!!")
      return True
    else:
      return False

The below 2 functions returns intersection of two lines given end point of each line segment, if the intersection lies between the segment

In [ ]:
def sort_points(points):
    y2 = [(points[i][0],points[i][1]) for i in range(len(points))]

    if len(set(y2))==2:
        return list(set(y2))

    x = [point[0] for point in points]
    y = [point[1] for point in points]
    x = sorted(x)
    y = sorted(y)

    req_points = []
    if (x[1], y[1]) in points:
        req_points.append((x[1], y[1]))
        req_points.append((x[2], y[2]))
    elif (x[1], y[2]) in points:
        req_points.append((x[1], y[2]))
        req_points.append((x[2], y[1]))

    return req_points

In [ ]:
def return_intersect(point1,point2,point3,point4):
    a1,b1,c1 = line(point1,point2)
    a2,b2,c2 = line(point3,point4)
    d = a1*b2 - a2*b1
    dx = c1*b2 - c2*b1
    dy = a1*c2 - a2*c1
    all1 = [point1,point2,point3,point4]
    if d==0 and dx==0 and dy==0:
        points = sort_points(all1)
        t = {0: points[0], 1: points[1]}
        return t
    elif d!=0:
        x,y = dx/d,dy/d
        if (min(point1[0],point2[0])<= x <= max(point1[0],point2[0])) and \
        (min(point3[0],point4[0])<= x <= max(point3[0],point4[0])) and \
        (min(point1[1],point2[1])<= y <= max(point1[1],point2[1])) and \
        (min(point3[1],point4[1])<= y <= max(point3[1],point4[1])):
            return x,y
    else:
        return None

Checks whether a point is inside a quadilateral or not

In [ ]:
def return_if_inside(point,boundary):
    p = [point]*4
    return overlap_check(boundary,p)

Returns end points of a quadilateral in order given the points in random order 

In [ ]:
def return_in_order(random):
    initial = random[0]
    ordered = [initial]
    list_t = random.copy()
    list_t.remove(initial)
    n = 0
    while True:
        i = list_t[n]

        temp_list = random.copy()
        temp_list.remove(i)
        if initial in temp_list:
            temp_list.remove(initial)

        a,b,c = line(initial,i)
        values = []

        for j in temp_list:
            temp = a*j[0] + b*j[1] - c
            values.append(temp)

        if min(values)*max(values)>=0:
            initial = i
            ordered.append(i)
            if len(ordered) == len(random):
                break
            n=-1
            list_t.remove(i)
        n+=1

    return ordered

Returns the intersection point(s) of 2 quadilaterals if they intersect

In [ ]:
def intersect_dect(ohrc_csv_file,tmc_csv_file):
    ohrc_boundary = bound(ohrc_csv_file)
    tmc_boundary = bound(tmc_csv_file)
    points = []
    if overlap_check(ohrc_boundary,tmc_boundary):
        for i in range(4):
            if i==3:
                a,b = 3,0
            else:
                a,b = i,i+1
            for j in range(4):
                if j==3:
                    c,d = 3,0
                else:
                    c,d = j,j+1

                a1 = return_intersect(ohrc_boundary[a],ohrc_boundary[b],tmc_boundary[c],tmc_boundary[d])
                if type(a1) == type({}):
                    k = list(a1.values())
                    point1 = k[0]
                    point2 = k[1]
                    points.append(point1)
                    points.append(point2)
                elif a1 is not None:
                    points.append(a1)


        for i in tmc_boundary:
            if return_if_inside(i,ohrc_boundary):
                points.append(i)

        for i in ohrc_boundary:
            if return_if_inside(i,tmc_boundary):
                points.append(i)

        points.append(points[0])
        lr,ur = get_maximal_rectangle(points)
        points = rect2poly(lr,ur)
        points = points[:len(points)-1]

        return points

    else:
        return None

Plots the quadilateral given its boundary points

In [ ]:
def plotting(boundary):
    plt.plot((boundary[0][0],boundary[1][0]),(boundary[0][1],boundary[1][1]),c='b')
    plt.plot((boundary[1][0],boundary[2][0]),(boundary[1][1],boundary[2][1]),c='b')
    plt.plot((boundary[2][0],boundary[3][0]),(boundary[2][1],boundary[3][1]),c='b')
    plt.plot((boundary[3][0],boundary[0][0]),(boundary[3][1],boundary[0][1]),c='b')

    plt.show()

Returns the scan and pixel values of tmc and ohrc files whereever they intersect if at all they intersect, which are needed for slicing the image

In [ ]:
def return_p_s(ohrc_csv1,tmc_csv1):
    ohrc_csv1 = ohrc_csv1
    tmc_csv1 = tmc_csv1
    boundary = intersect_dect(ohrc_csv1,tmc_csv1)
    ohrc_s_p = {}
    tmc_s_p = {}
    if boundary is not None:

      for i in boundary:
          i = tuple(i)
          temp_o = ohrc_csv1.copy(deep=True)
          temp_t = tmc_csv1.copy(deep=True)
          temp_o['Longitude'] = temp_o['Longitude'] - i[0]
          temp_o['Latitude'] = temp_o['Latitude'] - i[1]
          temp_t['Longitude'] = temp_t['Longitude'] - i[0]
          temp_t['Latitude'] = temp_t['Latitude'] - i[1]
          temp_o["new"] = np.abs(temp_o['Longitude']) + np.abs(temp_o['Latitude'])
          temp_t["new"] = np.abs(temp_t['Longitude']) + np.abs(temp_t['Latitude'])

          min_o = temp_o.new.min()
          min_t = temp_o.new.min()

          temp_o1 = temp_o[temp_o.new==min_o]
          temp_t1 = temp_t[temp_t.new==min_t]

          scan_o = temp_o1['Scan'].item()
          pixel_o = temp_o1['Pixel'].item()

          scan_t = temp_t1['Scan'].item()
          pixel_t = temp_t1["Pixel"].item()

          ohrc_s_p[i] = (pixel_o,scan_o)
          tmc_s_p[i] = (pixel_t,scan_t)

      ohrc_s_p1 = list(ohrc_s_p.values())
      tmc_s_p1 = list(tmc_s_p.values())
      ohrc_s_p1.append(ohrc_s_p1[0])
      tmc_s_p1.append(tmc_s_p1[0])

      lr,ur = get_maximal_rectangle(ohrc_s_p1)
      ohrc_s_p1 = rect2poly(lr,ur)
      ohrc_s_p1 = ohrc_s_p1[:len(ohrc_s_p1)-1]

      lr,ur = get_maximal_rectangle(tmc_s_p1)
      tmc_s_p1 = rect2poly(lr,ur)
      tmc_s_p1 = tmc_s_p1[:len(tmc_s_p1)-1]

      ohrc_s_p1 = list(np.round(np.array(ohrc_s_p1)).astype('int'))
      tmc_s_p1 = list(np.round(np.array(tmc_s_p1)).astype('int'))
      return ohrc_s_p1,tmc_s_p1
    else:
      return None

In [ ]:
l = []
for _,j in df_tmc.iterrows():
  t_csv = pd.read_csv('/content/drive/MyDrive/TMC_Files/Tmc_data/'+j.new_path+'/geometry/calibrated/'+j.Number[0]+'/'+j.csv_path)
  for _,i in df_ohrc.iterrows():
    o_csv = pd.read_csv('/content/drive/MyDrive/OHRC_Files/ohrc_files/'+i.New_Path2+'/geometry/calibrated/'+i.Number[0]+'/'+i.csv_path)
    try:
      a = return_p_s(o_csv,t_csv)
      if a is not None:
        l.append(a)
        print('OverLap!')
    except:
      l.append(None)
print(l)